In [0]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time

In [263]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
url = "https://github.com/facebookresearch"
r = requests.get(url)

In [0]:
soup1 = BeautifulSoup(r.content, 'html5lib') 

coverpage = soup1.find_all('h3', class_='wb-break-all')
starspage = soup1.find_all('a', class_="no-wrap muted-link mr-3", href=re.compile("stargazers"))


In [0]:
news=[]
titles=[]
links=[]
stars=[]
description = []
for n in np.arange(0, 30): 
    link = "https://github.com" + coverpage[n].find('a')['href']
    links.append(link)

    title = coverpage[n].find('a').get_text()
    titles.append(title)

    r2 = requests.get(link)
    soup2 = BeautifulSoup(r2.content, 'html5lib')
    body = soup2.find_all('article', class_='markdown-body entry-content')
    desc = soup2.find_all('title')

    list_paragraphs = []
    for p in np.arange(0, len(body)):
        paragraph = body[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        description.append(desc[p].get_text())
        

    final_article = re.sub("\\xa0", "", final_article)
        
    news.append(final_article)
    
for i in np.arange(0,30):
    titles[i] = re.sub("\\n", "", titles[i])
    news[i] = re.sub("\\n", "", news[i])

In [0]:
star=[]
for i in range(0, 20):
  star.append(starspage[i].get_text())
  star[i] = re.sub("\\n", "", star[i])
  star[i] = re.sub("[,]", "", star[i])
  star[i] = float(star[i].strip())


stars = pd.DataFrame(
    {'Saved' : star
})

In [0]:
features = pd.DataFrame(
     {'Article Content': news,
      'Article Title' : titles,
      'Article Link' : links
    })
df = pd.DataFrame(
    {'Content': news
     })
df1 = pd.DataFrame(
    {'Desc':description})


In [269]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess(con):
    con = [REPLACE_NO_SPACE.sub("", line.lower()) for line in con]
    con = [REPLACE_WITH_SPACE.sub(" ", line) for line in con]
    
    return con

Processed_Content = preprocess(news)
Processed_Description = preprocess(description)

In [0]:

english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for text in corpus:
        removed_stop_words.append(
            ' '.join([word for word in text.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words = remove_stop_words(Processed_Content)
no_stopwords_desc = remove_stop_words(Processed_Description)

In [0]:
from textblob import TextBlob as tb

In [0]:
df2=[]

i = 0
for text in Processed_Content:
    prediction = {}
    desc = Processed_Description[i]
    words = word_tokenize(text)
    text = text
    rate_nostop = len(no_stop_words)/len(words)

    def avgtoken(words):
      length=[]
      for j in words:
          length.append(len(j))
      return np.average(length)


    pos,neg = [],[]
    pol_pos,pol_neg = [],[]
    for j in words:
        analysis = tb(j)
        polarity = analysis.sentiment.polarity
        if polarity>0:
            pos.append(j)
            pol_pos.append(polarity)
        elif polarity<0:
            neg.append(j)
            pol_neg.append(polarity)
        
    prediction['pos_rate'] = len(pos)/len(text)
    prediction['neg_rate'] = len(neg)/len(text)
    prediction['min_pp'] = min(pol_pos)
    prediction['max_pp'] = max(pol_pos)
    prediction['avg_pp'] = np.average(pol_pos)
    if pol_neg!=[]:
        prediction['min_np'] = min(pol_neg)
        prediction['max_np'] = max(pol_neg)
        prediction['avg_np'] = np.average(pol_neg)

    param = 0
    if ("ai" or "artificial intelligence" in desc) or ("ai" or "artificial intelligence" in text):
        param+=1
        param+=1
    if ("nlp" or "nlu" or "natural language processing" in desc) or ("nlp" or "nlu" or "natural language processing" in text):
        param+=1
    if ("nlg" or "natural language generation" in desc) or ("nlg" or "natural language generation" in text):
        param+=1
    if ("robot" in desc) or ("robot" in text):
        param+=1
    if ("cognitive" or "cognition" in desc) or ("cognitive" or "cognition" in text):
        param+=1
    if ("rl" or "reinforcement learning" in desc) or ("rl" or "reinforcement learning" in text):
        param+=1
        param+=1
    if ("dl" or "deep learning" in desc) or ("dl" or "deep learning" in text):
        param+=1
    if ("speech recognition" in desc) or ("speech recognition" in text):
        param+=1
    if ("3d" in desc) or ("3d" in text):
        param+=1
    prediction['total_param'] = param
    df2.append(prediction)
    i+=1


In [0]:
pred_info = pd.DataFrame(df2)

In [0]:
result = pred_info.dropna()
features=features.drop([2,3,22])
stars = stars.drop([2,3])

In [0]:
x_train = result.iloc[0:18,:]
y_train = stars.iloc[0:18,:]
x_test = result.iloc[18:,:]
title_final = features.iloc[18:,:]

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [278]:
rfr = RandomForestRegressor(random_state=42)
rfr.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [0]:
final = pd.DataFrame({
    
    'Title': title_final['Article Title'],
    'Virality': rfr.predict(x_test), 
    })

In [286]:
final

,Title,Virality
20,hiplot,2697.19
21,poincare-embeddings,5113.35
23,classifier-balancing,1617.83
24,GraphLog,2931.96
25,ActivityNet-Entities,10751.21
26,higher,2535.78
27,nds,4611.85
28,EGG,2243.81
29,fvcore,4475.80
